# Scan Cluster
Scan Clusters is a program to look for homologous clusters in bacterial (but could also work on eukaryotes and viruses) genomes. If only a query cluster is provided the program will build HMMs for each gen in the cluster and then it will use hmmsearch to find the homologous genes in all the subject genomes. Homologous protein searches can be done in remote or local mode. For the remote mode, Scan_cluster will try to run blastp _-remote_ searches to retrieve homologous proteins. Then from the filtered blast results it will make the HMMs. In local mode, it will use a provided blast database or make a local Blast database from all the genomes to be analyzed, and run local blastp searches. From these searches the HMMs will be constructed. Alternatively, scan_cluster can search for clusters containing a predefined set of HMM. Finally, it can search for homologous of the proteins in the query cluster using blastp instead of hmmsearch.

A cluster is then defined if there are multiple query protein hits in the same region of the target genome.
The following criteria is used for cluster definition:
*n_prots_between argument specifies the maximum number of proteins allowed between two consecutive genes in the query cluster.
*max_alien_prots argument specifies the maximum number of proteins in the target cluster that are not present in the query cluster.
*min_target_prots argument specifies the minimum of query proteins required to be found in target cluster. Default=3.)
*min_cluster_coverage argument specifies the minimum of cluster coverage (proportion). Default=.5. The program will use as minimum half of the query proteins.

The best clusters are then compared by pairwise blastp and a multiple cluster alignment (MCA) is generated by a progressive algorithm using dynamic programming. The scoring scheme for the MCA uses the orientation and pairwise identity values between all proteins in the clusters. Gaps are introduced when alien genes are present in the target clusters or when query genes are missing.
The guide tree used in the MCA is reported jointly with an annotation file for ITOL.
The clusters are also exported in genbank format to perform the cluster analysis with _Clinker_ software.

![algorithm](https://github.com/maurijlozano/scan_cluster/blob/main/algorithm.png?raw=true)

## Command line arguments
```
usage: scan_cluster.py [-h] [-Q QFILE] [-R REPID] [-s CSTART] [-e CEND]
                       [-q QCLUSTER] [--Reference REFGENOME] [-f HMM_FOLDER]
                       [-S SFILE] [-F SFOLDER] [-E GBEXT] [-o RES_FOLDER]
                       [--overwrite] [--only_blastp] [-n PROTS_BETWEEN]
                       [-M MAX_ALIEN_PROTS] [-m MIN_TARGET_PROTS]
                       [--min_cluster_coverage MIN_CLUSTER_COVERAGE] [-g GAP]
                       [--mismatch_score MISMATCH]
                       [--local_blast_db LOCAL_BLAST_DB] [--Generate_local_db]
                       [--Blast_DB BLASTP_DATABASE] [--Blast_evalue EVALUE]
                       [--Blast_max_targets MAX_TARGET] [--Blast_qcov QCOV]
                       [--Blast_scov SCOV] [--hmm_evalue HMM_EVALUE]
                       [--hmm_cover HMM_COVER]

This program was designed to identify orthologous genes clusters.

options:
  -h, --help            show this help message and exit

Query cluster:
  -Q QFILE, --QueryFile QFILE
                        Query genome in Genbank format.
  -R REPID, --Replicon_ID REPID
                        Required for draft and multireplicon genomes.
  -s CSTART, --cluster_start CSTART
                        Genome cluster start location.
  -e CEND, --cluster_end CEND
                        Genome cluster end location.

To search or a cluster provided in Genbank format:
  -q QCLUSTER, --QueryCluster QCLUSTER
                        Query cluster in Genbank format.
  --Reference REFGENOME
                        Specifies the genome Gb file to be used as reference.

To search or a cluster with a defined set of protein HMMs:
  -f HMM_FOLDER, --hmm_folder HMM_FOLDER
                        Folder containing the HMM profiles for the proteins to
                        include in the cluster.

Target Genomes:
  -S SFILE, --SubjectFile SFILE
                        Subject genome in Genbank format.
  -F SFOLDER, --SubjectFolder SFOLDER
                        Folder containing the subject genomes in Genbank
                        format.
  -E GBEXT, --Genbank file extension GBEXT
                        Genbank file extension. Default .gb

Output Folder:
  -o RES_FOLDER, --Results_folder RES_FOLDER
                        Results folder name.
  --overwrite           Overwrite previous results.

Running mode:
  --only_blastp         Runs using only blast for the identification of
                        homolog proteins.

Cluster definition arguments:
  -n PROTS_BETWEEN, --n_prots_between PROTS_BETWEEN
                        Maximum number of proteins allowed between two
                        consecutive genes in the query cluster. Default = half
                        of proteins in the cluster
  -M MAX_ALIEN_PROTS, --max_alien_prots MAX_ALIEN_PROTS
                        Maximum number of proteins in the target cluster that
                        are not present in the query cluster. Default = not
                        limited (Number of proteins in cluster * 3).
  -m MIN_TARGET_PROTS, --min_target_prots MIN_TARGET_PROTS
                        Minimum of query proteins required to be found in
                        target cluster. Default=3.)
  --min_cluster_coverage MIN_CLUSTER_COVERAGE
                        Minimum of cluster coverage, proportion. Default=.5.
                        The program will use as minimum half of the query
                        proteins. If you are running only with HMMs, this
                        value should be the fraction of the HMM required in a
                        cluster.)
  -g GAP, --gap_penalty GAP
                        Gap penalty for cluster alignment. Default = 10
  --mismatch_score MISMATCH
                        Mismatch score for cluster alignment. Alignment of
                        genes that are not orthologs are penalized. Default =
                        20

Blast and HMMSearch options:
  --local_blast_db LOCAL_BLAST_DB
                        A local blastp database generated with makeblastdb
                        program... <Folder name>
  --Generate_local_db   A local blastp database will be generated from the
                        proteome of all the analyzed subject sequences...
  --Blast_DB BLASTP_DATABASE
                        Database for remote blastp, used to retrieve homologs
                        for HMM generation. Default=nr Available: nr,
                        refseq_select, refseq_protein, landmark, swissprot,
                        pataa, pdb, env_nr, tsa_nr
  --Blast_evalue EVALUE
                        E-value cut-off for remote blastp, used to retrieve
                        homologs for HMM generation.
  --Blast_max_targets MAX_TARGET
                        Maxímum number of targets for Blastp search.
                        Default=250
  --Blast_qcov QCOV     Query coverage percent for Blastp search. Default=45
  --Blast_scov SCOV     Subject coverage percent for Blastp search. Default=45
  --hmm_evalue HMM_EVALUE
                        E-value cut-off for hmmsearch. Default = 0.00001
  --hmm_cover HMM_COVER
                        HMM coverage cut-off for hmmsearch. Default = 45

```

Running in the default mode can be very slow, depending on the NCBI blast server load (runs -remote blastp searches).
Sequence coverages are set to 45% to improve cluster detection. Using 70% of query coverage some clusters were not found.
Also the maximum number of alien proteins (-M MAX_ALIEN_PROTS) and the maximum number of proteins allowed between two consecutive genes in the query cluster (-n PROTS_BETWEEN) should be adjusted for better sensibility.
To get more sensibility in the detection of distant clusters (with similar gene composition but different organization) the --only_blastp mode is not recommended. It should be used to gain speed only.

## Examples
### 1. Extract cluster from genbank file and search using remote blast to build HMM
```
./scan_cluster -Q <query genome gb> -R <Replicon ID as in the gb file> -s <int: start coordinate> -e <int: end coordinate> -F <Folder: folder with all the genomes to use as subject> -o <Folder: output folder>
```
### 2. Extract cluster from genbank file and search using local blast to build HMM
```
./scan_cluster -Q <query genome gb> -R <Replicon ID as in the gb file> -s <int: start coordinate> -e <int: end coordinate> -F <Folder: folder with all the genomes to use as subject> -o <Folder: output folder> --Generate_local_db
```

### 3. Search for clusters using HMMs provided by the user
```
./scan_cluster -f <HMM folder: folder with the HMM for genes in the cluster> -F <Folder: folder with all the genomes to use as subject> -o <Folder: output folder>
```

#### 4. Search using a genbank file only containing the cluster of interest and using the blast only mode
```
./scan_cluster -q <genbank cluster file>  -F <Folder: folder with all the genomes to use as subject> --only_blastp -o <Folder: output folder>
```


# Install scan_cluster

In [ ]:
#@markdown ## Install the required python modules, blast and hmmer.

print('Installing the requirements for Scan cluster...')
import os,sys
from sys import platform
from shutil import which
import subprocess
if not os.path.exists('requirements.txt'):
  !wget https://github.com/maurijlozano/scan_cluster/raw/refs/heads/main/requirements.txt
  os.system('pip install -r requirements.txt')

if not os.path.exists('scan_cluster.py'):
  !wget https://github.com/maurijlozano/scan_cluster/raw/refs/heads/main/scan_cluster.py


if not platform.startswith("linux"):
    print('This program only works in linux systems...\n')

def is_installed(program):
    """Check whether `program` is on PATH and marked as executable."""
    return (which(program) != None)

!sudo apt-get update  >/dev/null 2>&1
!sudo apt-get upgrade -y  >/dev/null 2>&1


if not is_installed('blastp'):
    os.system('sudo apt install ncbi-blast+')
    print(f'Blast is installed: {which("blastp")}')
else:
    print(f'Blast is installed: {which("blastp")}')

if not is_installed('hmmsearch'):
    os.system('sudo apt install hmmer')
    print(f'HMMER is installed: {which("hmmsearch")}')
else:
    print(f'HMMER is installed: {which("hmmsearch")}')

if not is_installed('mafft'):
    os.system('sudo apt install mafft')
    print(f'mafft is installed: {which("mafft")}')
else:
    print(f'mafft is installed: {which("mafft")}')


if not os.path.exists('./datasets'):
  print('Instalando Datasets')
  !wget https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/v2/linux-amd64/datasets 2> /dev/null 1>> logs.txt
  !chmod +x datasets
else:
  print('Datasets already installed...')


if os.path.exists('scan_cluster.py'):
    os.system('chmod +x scan_cluster.py')
    print(f'\n\nTesting scan_cluster')
    process = subprocess.run([f'./scan_cluster.py','-h'], capture_output=True, text=True)
    print(process.stdout)
else:
    print('scan_cluster.py not found in current folder.')

scan_cluster = './scan_cluster.py'
datasets = './datasets'


# Download genomes from NCBI

In [ ]:
#@markdown # Query Cluster
#@markdown Download refernce genome for cluster definition / Upload cluster in Genebank format
from google.colab import files


select_query_mode = 'NCBI accession' #@param ['NCBI accession', 'Upload']

#@markdown <br>

#@markdown **NCBI download mode**

#@markdown Required: accession number of the genome containing the query cluster, replicon id, and start and end coordinates.
reference_genome_accession = '' #@param {type: 'string'}
acc = reference_genome_accession

replicon = '' #@param {type:'string'}
cluster_start = -1 #@param {type:'integer'}
cluster_end = -1 #@param {type:'integer'}

if not os.path.exists('/content/genomes'):
  os.mkdir('/content/genomes')

try:
  if (select_query_mode == 'NCBI accession') and (len(acc) > 0):
    if not os.path.exists(f'/content/genomes/{acc}.gb'):
      print(f'Downloading {acc} genome...')
      try:
        !./datasets download genome accession {acc} --include gbff --filename {acc}.zip >/dev/null 2>&1
        !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/genomic.gbff -d ./ >/dev/null 2>&1
        os.remove(f'{acc}.zip')
        !mv genomic.gbff /content/genomes/{acc}.gb >/dev/null 2>&1
        refgenome = f'/content/genomes/{acc}.gb'
        print(f'The reference genome ({acc}) was downloaded to {refgenome}...')
      except:
        print(f'Error: {acc} not found')
    else:
      refgenome = f'/content/genomes/{acc}.gb'
      print(f'The reference genome ({acc}) was already downloaded...')
  elif select_query_mode == 'Upload':
    uploaded_files = files.upload()
    print('Uploaded Cluster:', end = '')
    refgenome = list(uploaded_files.keys())[0]
    print(f'{refgenome} ', end = '')
  else:
    print('No genomes to download. Species name or Assembly accession list not provided.')
except:
  print('Error: Something went wrong')

In [ ]:
#@markdown #Download Target genomes
#@markdown **Input**: a _genus_ _species_ name, a comma separated list of NCBI assembly accession numbers or upload your genomes
#get accessions or sp

import shutil, re

select_mode = 'Genus species' #@param ['NCBI accessions', 'Genus species', 'Upload']
get_sp = False
get_acc = False
upload = False

if select_mode == 'NCBI accessions':
  get_acc = True
elif select_mode == 'Genus species':
  get_sp = True
elif select_mode == 'Upload':
  upload = True

#@markdown <br>

#@markdown ##Get genomes by genus and species.
#@markdown Input a comma separated list of the _genus species_ to download

get_species_genomes = '' #@param {type: 'string'}
get_species_genomes = re.sub(', ',',',get_species_genomes)

get_species_genomes = get_species_genomes.split(',')

#assembly level filter
assembly_level = True #@param {type: 'boolean'}
if assembly_level:
  assembly_level_flag = '--assembly-level complete'
else:
  assembly_level_flag = ""

#Annotated
annotated = True #@param {type: 'boolean'}
if annotated:
  annotated_flag = '--annotated'
else:
  annotated_flag = ""

#exclude-atypical
exclude_atypical = True #@param {type: 'boolean'}
if exclude_atypical:
  exclude_atypical_flag = '--exclude-atypical'
else:
  exclude_atypical_flag = ""

#refseq only
refseq_only = True #@param {type: 'boolean'}
if refseq_only:
  refseq_only_flag = '--assembly-source RefSeq'
else:
  refseq_only_flag = ""

#@markdown <br>

#@markdown ### Use if you want to download from NCBI a list of accession numbers.
#@markdown Input: list of comma separated accession numbers separated by spaces.
genome_accessions = '' #@param {type: 'string'}
genome_accessions = re.sub(' ','',genome_accessions)
genome_accessions_list = genome_accessions.split(',')



import os, glob

if not os.path.exists('/content/genomes'):
  os.mkdir('/content/genomes')
else:
  files_in_folder = glob.glob('/content/genomes/*.gb')
  print(f'Genomes folder already exists. Genomes already downloaded:')
  for f in files_in_folder:
    print(f'{f}')
  print(f'Scan_cluster uses all the genomes in /content/genomes/ as target genomes. Pleas delete all the unwanted genomes.')


genome_files = []

try:
  if get_sp and (len(get_species_genomes) > 0):
    for sp in get_species_genomes:
      print(f'Downloading {sp} genomes...')
      try:
        !./datasets download genome taxon "{sp}" --include gbff {assembly_level_flag} {exclude_atypical_flag } {refseq_only_flag} >/dev/null 2>&1
        !unzip -o ncbi_dataset >/dev/null 2>&1
        genome_list = glob.glob('/content/ncbi_dataset/data/*/**/*.gbff', recursive=True)
        base_names = {os.path.split(os.path.split(i)[0])[1]:i for i in genome_list}
        for i in base_names.keys():
          gfile = f'/content/genomes/{i}.gb'
          !mv {base_names[i]} {gfile}
          genome_files.append(gfile)
        os.remove(f'/content/ncbi_dataset.zip')
        shutil.rmtree('/content/ncbi_dataset')
      except:
        print(f'--> Error: {sp} not found. Please verify the organism name.')
  elif get_acc and len(genome_accessions_list) > 0 :
    for acc in genome_accessions_list:
      if not os.path.exists(f'/content/genomes/{acc}.gb'):
        print(f'Downloading {acc} genome...')
        try:
          !./datasets download genome accession {acc} --include gbff --filename {acc}.zip >/dev/null 2>&1
          !unzip -o -j {acc}.zip ncbi_dataset/data/{acc}/genomic.gbff -d ./ >/dev/null 2>&1
          os.remove(f'{acc}.zip')
          !mv genomic.gbff /content/genomes/{acc}.gb >/dev/null 2>&1
          genome_files.append(f'/content/genomes/{acc}.gb')
        except:
          print(f'Error: {acc} not found')
      else:
        genome_files.append(f'/content/genomes/{acc}.gb')
        print(f'The reference genome ({acc}) was already downloaded...')
  elif upload:
    uploaded_files = files.upload()
    print('Uploaded genomes:', end = '')
    genome_files = list(uploaded_files.keys())
    for gn in genome_files:
      print(f'{gn} ', end = '')
  else:
    print('No genomes to download. Genus species name, assembly accession list or upload your files.')
except:
  print('Error: Something went wrong')

print('Done!')
genome_files = list(set(genome_files))

In [ ]:
#@markdown ###Use this block if you want to zip and download all the genomes to a local computer
import os
import shutil
from google.colab import files
zip_downoad = False #@param {type:'boolean'}

if zip_downoad:
  if not os.path.exists('/content/genomes.zip'):
    !zip -r /content/genomes.zip /content/genomes
  files.download("/content/genomes.zip")

In [ ]:
#@markdown ##Generate a table with information of the downloaded genomes
from Bio import SeqIO
import re, glob

target_genome_files = glob.glob('/content/genomes/*.gb')
with open('genomes.csv','w') as tf:
  tf.write('Assembly Accession number,Organism,Taxonomy,Description,Comments\n')
  for gnome in target_genome_files:
    acc = os.path.splitext(os.path.basename(gnome))[0]
    r = list(SeqIO.parse(gnome,'gb'))[0]
    if len(r.annotations['organism']) > 0:
      org = r.annotations['organism']
    else:
      org = ''
    if len(r.annotations['taxonomy']) > 0:
      tax = '; '.join(r.annotations['taxonomy'])
    else:
      tax = ''
    desc = re.sub(',',';',r.description)
    desc = re.sub(' ?[Cc][Oo][Nn][Tt][Ii][Gg].*','',desc)
    desc = re.sub(' ?[Nn][Oo][Dd][Ee].*','',desc)
    desc = re.sub(' ?[Ss][cC][aA][Ff].*','',desc)
    desc = re.sub(' ?[Pp][Ll][aA][Ss].*','',desc)
    desc = re.sub(' ?[Cc][Hh][Rr][Oo].*','',desc)
    if 'comment' in r.annotations.keys():
      if len(r.annotations['comment']) > 0:
        comments = re.sub('[,]','; ',r.annotations['comment'])
        comments = re.sub('\n',' ',comments)
    else:
      comments = ''
    tf.write(f'{acc},{org},{tax},{desc},{comments}\n')

from google.colab import files
download_table = False #@param {type:'boolean'}

if download_table:
  files.download('genomes.csv')

import pandas as pd
table = pd.read_csv('genomes.csv')
table

# Run scan_cluster

In [ ]:
#@markdown Run with the default options in local mode and defining a cluster by its genomic coordinates in the reference genome.
select_running_mode = 'default' #@param ['default','hmm','command_line']
hmm_folder =  '' #@param {type:'string'}
overwrite = True #@param {type:'boolean'}
if overwrite:
  overwrite_flag = '--overwrite'
else:
  overwrite_flag = ''
#@markdown <br>


#@markdown ###Additional Options: for default values use -1
#@markdown Maximum number of proteins allowed between two consecutive genes in the query cluster. Default = half of proteins in the cluster"
n_prots_between = -1 #@param {type:'integer'}
if n_prots_between > 0:
  n_prots_between_flag = f'-n {n_prots_between}'
else:
  n_prots_between_flag = ''

#@markdown Maximum number of proteins in the target cluster that are not present in the query cluster. Default = not limited
max_alien_prots = -1 #@param {type:'integer'}
if max_alien_prots > 0:
  max_alien_prots_flag = f'-M {max_alien_prots}'
else:
  max_alien_prots_flag = ''

#@markdown Minimum of query proteins required to be found in target cluster.
min_target_prots = -1 #@param {type:'integer'}
if min_target_prots > 0:
  min_target_prots_flag = f'-m {min_target_prots}'
else:
  min_target_prots_flag = ''

#@markdown Minimum of cluster coverage, proportion. Default=.5. The program will use as minimum half of the query proteins. If you are running only with HMMs, this value should be the fraction of the HMM required in a cluster.
min_cluster_coverage = -1 #@param {type:'number'}
if min_cluster_coverage > 0:
  min_cluster_coverage_flag = f'--min_cluster_coverage {min_cluster_coverage}'
else:
  min_cluster_coverage_flag = ''

#@markdown Homologous search coverage [ For both blastp and hmmsearch ]
min_coverage = 45 #@param {type:'integer'}
if min_coverage > 0:
  min_coverage_flag = f'--Blast_qcov {min_coverage} --Blast_scov {min_coverage} --hmm_cover {min_coverage}'
else:
  min_coverage_flag = ''

#@markdown <br>
cluster_found = False

if select_running_mode == 'default':
  print(f'Running with default options. \nRef genome: {refgenome}\nReplicon: {replicon}\nCluster start: {cluster_start}\nCluster end: {cluster_end}')
  if not select_query_mode == 'Upload' :
    if os.path.exists(refgenome):
      for r in SeqIO.parse(refgenome,'gb'):
        if r.id == replicon:
          if (cluster_start > 0) and (cluster_end > 0):
            cluster = r[cluster_start:cluster_end]
            for f in cluster.features:
              if (f.type == 'CDS') and ('locus_tag' in f.qualifiers) and ('product' in f.qualifiers):
                print(f"{f.qualifiers['locus_tag'][0]} - {f.qualifiers['product'][0]}")
            cluster_found = True
            break
  else:
    cluster_found = True
  if cluster_found:
    if select_query_mode == 'Upload' :
      !./scan_cluster.py -q {refgenome} -F /content/genomes/ --Generate_local_db {overwrite_flag} {n_prots_between_flag} {max_alien_prots_flag} {min_target_prots_flag} {min_cluster_coverage_flag} {min_coverage_flag}
    else:
      !./scan_cluster.py -Q {refgenome} -R {replicon} -s {cluster_start} -e {cluster_end} -F /content/genomes/ --Generate_local_db {overwrite_flag} {n_prots_between_flag} {max_alien_prots_flag} {min_target_prots_flag} {min_cluster_coverage_flag} {min_coverage_flag}
  else:
    print('Cluster not found')
elif select_running_mode == 'command_line':
  print(f'Running with command line mode.')
  #@markdown ###For customized options run the command line mode:
  cl = '' #@param {type:'string'}
  !{cl}
elif select_running_mode == 'hmm':
  print(f'Running with hmm mode.')
  if len(hmm_folder) > 0:
    if os.path.exists(f'{hmm_folder}'):
      hmms = glob.glob(f'{hmm_folder}/*.hmm')
      if len(hmms) > 0:
        print(f'Found {len(hmms)} files in {hmm_folder}')
        !./scan_cluster.py -f {hmm_folder} -F /content/genomes/ {overwrite_flag} {n_prots_between_flag} {max_alien_prots_flag} {min_target_prots_flag} {min_cluster_coverage_flag} {min_coverage_flag}
      else:
        print(f'No hmm files found in {hmm_folder}')
    else:
      print(f'No hmm folder: {hmm_folder}. ')
  else:
    print('A folder with the HMM files is required...')



In [ ]:
#@markdown ## Ascii draw tree
from Bio import Phylo
import os
import re

tree_file = '/content/Results/tree.nhx' #@param {type:'string'}
if os.path.exists(tree_file):
  tree = Phylo.read(tree_file,'nexus')
  if os.path.exists('genomes.csv'):
    table = pd.read_csv('genomes.csv')
    rename = {row['Assembly Accession number']:row['Description'].split(';')[0] for _,row in table.iterrows() }
    for leaf in tree.get_terminals():
      name = leaf.name.split('__')[0]
      rep = leaf.name.split('__')[1]
      coords = leaf.name.split('__')[2]
      if name in rename.keys():
        leaf.name = re.sub(':','',f'{rename[name]}__{rep}__{coords}')
  Phylo.draw_ascii(tree)
  Phylo.write(tree,'tree.nwk','newick')
else:
  print(f'{tree_file} not found')

import re
with open('tree.nwk','r') as infile, open('tmp.txt','w') as outfile:
  for line in infile:
    line = re.sub("\\'","",line)
    outfile.write(line)
!mv tmp.txt tree.nwk


iTolAnnotation = '/content/Results/iTOLData.txt' #@param {type:'string'}
if os.path.exists(iTolAnnotation):
  with open(iTolAnnotation,'r') as r, open('iTolData_rn.txt','w') as w:
    for line in r:
      if 'GCF' in line:
        a=line.split(',')[0].split('__')
        a = f'{rename[a[0]]}__{a[1]}__{a[2]}'
        a = re.sub(':','',a)
        b=','.join(line.split(',')[1:])
        w.write(f'{a},{b}')
      else:
        w.write(line)

